<h1><center> OCT PROJET 


In [15]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
import glob


In [16]:

# --- Configuration ---
BASE_DATA_PATH = "./"
IMAGE_DIR = os.path.join(BASE_DATA_PATH, "Rad+cubeOD\OD\RAD") # Adaptez
OUTPUT_DIR = os.path.join(BASE_DATA_PATH, "rad_cropped") # Dossier pour les images recadrées
os.makedirs(OUTPUT_DIR, exist_ok=True)

# --- Coordonnées de recadrage (À DÉTERMINER PAR INSPECTION VISUELLE) ---
# Ces valeurs sont des EXEMPLES et devront être AJUSTÉES PRÉCISÉMENT pour VOS images.
# Ouvrez une image dans un éditeur, trouvez le rectangle du B-scan.
# (x_start, y_start) est le coin supérieur gauche du B-scan.
# (x_end, y_end) est le coin inférieur droit du B-scan.
# Exemple basé sur la structure typique des images Heidelberg Spectralis:
# La partie gauche (fond d'oeil) est souvent carrée et sa largeur est similaire à la hauteur du B-scan.
# Les métadonnées sont en bas.

# Exemple de valeurs (VOUS DEVEZ LES AJUSTER !) :
# Supposons que l'image complète fasse 1024x496 (largeur x hauteur)
# et que la partie fond d'oeil à gauche fasse environ 300-350 pixels de large.
# et que les métadonnées en bas fassent 50-70 pixels de haut.
CROP_X_START = 500  # Exemple: à partir de quelle colonne x commence le B-scan
CROP_Y_START = 20   # Exemple: à partir de quelle ligne y commence le B-scan (pour ignorer le noir en haut)
CROP_X_END = 1520   # Exemple: jusqu'à quelle colonne x va le B-scan (pour ignorer marge/logo à droite)
CROP_Y_END = 450    # Exemple: jusqu'à quelle ligne y va le B-scan (pour ignorer les métadonnées en bas)


In [17]:

# --- Fonctions ---
def load_image(image_path):
    img = cv2.imread(image_path) # Charger en couleur au cas où pour l'affichage
    if img is None:
        print(f"Erreur : Impossible de charger l'image à {image_path}")
    return img

def crop_image(image, x_start, y_start, x_end, y_end):
    if image is None:
        return None
    # S'assurer que les coordonnées sont valides
    h, w = image.shape[:2]
    x_s = max(0, x_start)
    y_s = max(0, y_start)
    x_e = min(w, x_end)
    y_e = min(h, y_end)

    if x_s >= x_e or y_s >= y_e:
        print(f"Erreur de recadrage: coordonnées invalides ({x_s},{y_s}) à ({x_e},{y_e}) pour une image de taille {w}x{h}")
        return image # ou None
    
    cropped = image[y_s:y_e, x_s:x_e]
    return cropped

def display_comparison(original_img, cropped_img, title_original="Originale", title_cropped="Recadrée"):
    if original_img is None or cropped_img is None:
        print("Une des images est None, impossible d'afficher.")
        return
        
    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    plt.imshow(cv2.cvtColor(original_img, cv2.COLOR_BGR2RGB))
    plt.title(title_original)
    plt.axis('off')

    plt.subplot(1, 2, 2)
    plt.imshow(cv2.cvtColor(cropped_img, cv2.COLOR_BGR2RGB))
    plt.title(title_cropped)
    plt.axis('off')
    
    plt.tight_layout()
    plt.show()


In [19]:
def crop_bscan():
    '''  Recadre les B-scans d'une série d'images en fonction des coordonnées définies'''

    # --- Script Principal ---
    image_files = glob.glob(os.path.join(IMAGE_DIR, "*.tif")) # Ou .png, .jpg
    if not image_files:
        print(f"Aucun fichier .tif trouvé dans {IMAGE_DIR}. Vérifiez le chemin et l'extension.")
        exit()

    # Traiter une seule image pour tester les coordonnées de recadrage
    test_image_path = image_files[0] # Prendre la première image
    print(f"Traitement de l'image test: {test_image_path}")

    full_image = load_image(test_image_path)

    if full_image is not None:
        # Afficher les dimensions pour aider à déterminer les coordonnées
        print(f"Dimensions de l'image complète: {full_image.shape} (hauteur, largeur, canaux)")
        
        # Appliquer le recadrage
        # VOUS DEVEZ AJUSTER CROP_X_START, CROP_Y_START, CROP_X_END, CROP_Y_END
        cropped_bscan = crop_image(full_image, CROP_X_START, CROP_Y_START, CROP_X_END, CROP_Y_END)

        if cropped_bscan is not None and cropped_bscan.size > 0 :
            print(f"Dimensions de l'image recadrée: {cropped_bscan.shape}")
            display_comparison(full_image, cropped_bscan, 
                            title_original=f"Originale ({os.path.basename(test_image_path)})", 
                            title_cropped=f"B-Scan Recadré (Coords: x[{CROP_X_START}:{CROP_X_END}], y[{CROP_Y_START}:{CROP_Y_END}])")
            
            # Optionnel: Sauvegarder l'image recadrée
            # output_filename = os.path.join(OUTPUT_DIR, "cropped_" + os.path.basename(test_image_path))
            # cv2.imwrite(output_filename, cropped_bscan)
            # print(f"Image recadrée sauvegardée ici : {output_filename}")
        else:
            print("Le recadrage a produit une image vide ou None.")
            if full_image is not None:
                plt.imshow(cv2.cvtColor(full_image, cv2.COLOR_BGR2RGB))
                plt.title("Image originale (échec du recadrage)")
                plt.show()


    # Une fois que vous êtes satisfait des coordonnées pour UNE image,
    # vous pouvez décommenter cette boucle pour traiter toutes les images :

    print("\n--- Traitement de toutes les images ---")
    for img_path in image_files:
        print(f"Traitement de : {img_path}")
        current_full_image = load_image(img_path)
        if current_full_image is not None:
            current_cropped_bscan = crop_image(current_full_image, CROP_X_START, CROP_Y_START, CROP_X_END, CROP_Y_END)
            if current_cropped_bscan is not None and current_cropped_bscan.size > 0:
                output_filename = os.path.join(OUTPUT_DIR, "cropped_" + os.path.basename(img_path))
                # Convertir en niveaux de gris avant de sauvegarder si vous ne voulez que le B-scan en gris
                # current_cropped_bscan_gray = cv2.cvtColor(current_cropped_bscan, cv2.COLOR_BGR2GRAY)
                # cv2.imwrite(output_filename, current_cropped_bscan_gray)
                cv2.imwrite(output_filename, current_cropped_bscan) # Sauvegarde en couleur (ou comme l'original)
                print(f"  -> Sauvegardée : {output_filename}")
            else:
                print(f"  -> Échec du recadrage pour {img_path}")


<h1> Segmentation de la rétine

In [ ]:
def main():

    gray_bscan = cv2.cvtColor(cropped_bscan, cv2.COLOR_BGR2GRAY)

    # Filtre Gaussien
    blurred = cv2.GaussianBlur(gray_bscan, (5, 5), 0) # Ajustez la taille du noyau

    # Ou Filtre Médian (bon pour le bruit poivre et sel)
    # blurred = cv2.medianBlur(gray_bscan, 5)

    # Filtre Gaussien
    blurred = cv2.GaussianBlur(gray_bscan, (5, 5), 0) # Ajustez la taille du noyau

    # Ou Filtre Médian (bon pour le bruit poivre et sel)
    # blurred = cv2.medianBlur(gray_bscan, 5)


def segment_ilm_gradient_based(image_processed):
    ilm_line = np.zeros(image_processed.shape[1], dtype=int)
    # Réduire la zone de recherche pour l'ILM (par exemple, le tiers supérieur de l'image)
    search_roi_y_end = image_processed.shape[0] // 3

    for x in range(image_processed.shape[1]):
        column = image_processed[:search_roi_y_end, x]
        
        # Calculer le gradient vertical (différence simple ou Sobel)
        # grad_y = np.diff(column.astype(float)) # Simple diff, longueur N-1
        # Ou utiliser Sobel pour un gradient plus robuste
        sobel_y = cv2.Sobel(column, cv2.CV_64F, 0, 1, ksize=3) # Gradient en Y

        # Trouver le premier pic positif important
        # On cherche une transition sombre -> clair, donc un gradient positif important
        # (ou un pic négatif si on utilise diff et qu'on regarde la valeur d'avant)
        
        # Simplification: trouver le premier pixel au-dessus d'un seuil après lissage
        # Cette partie est très similaire à la méthode simple mais sur une image mieux préparée
        # et avec un ROI. Une vraie détection de pic de gradient serait mieux.
        
        # Pour une approche "premier pixel au-dessus d'un seuil dans ROI" (plus simple à coder ici)
        threshold_val = np.mean(column) + np.std(column) # Seuil adaptatif simple
        potential_points = np.where(column > threshold_val)[0]

        if len(potential_points) > 0:
            ilm_line[x] = potential_points[0]
        else:
            # Gérer le cas où rien n'est trouvé (ex: prendre la moyenne des voisins)
            if x > 0:
                ilm_line[x] = ilm_line[x-1] 
            else:
                ilm_line[x] = search_roi_y_end // 2 # Estimation grossière
    return ilm_line